In [ ]:
#Kraabime veebilehelt PDFid ja slaidiesitlused maha

import os
import requests
from urllib.parse import urljoin
from bs4 import BeautifulSoup
import tika
from tika import parser
#Tegin siia meetodi, mis võtab nasdaqbalticu lehelt siis ISIN koodi ja kuupäeva järgi kõik .pdf formaadis failid,
#laeb need lokaalselt mulle alla.
def fetch_pdfs(ISIN,date):
    url = f"https://nasdaqbaltic.com/statistics/en/instrument/{ISIN}/reports?date={date}"
    #If there is no such folder, the script will create one automatically
    global folder_location 
    folder_location = ISIN
    basedir = r"C:\Users\marek.keskull\Documents\webscraping"
    fold = os.path.join(basedir, folder_location)
    print("The folder location that i am operating in: " + fold)
    if not os.path.exists(fold):os.mkdir(fold)

    response = requests.get(url)
    soup= BeautifulSoup(response.text, "html.parser")
    table = soup.find("tbody")
    #table_trs = table.find_all('tr')
    links = table.select("a[href*='con']")


    for link in links:
        filename = os.path.join(fold,link['href'].split('/')[-1])
        with open(filename, 'wb') as f:
            f.write(requests.get(urljoin(url,link['href'])).content)
 #kasutan tika libraryt, selle peab panema lokaalselt käima eraldi, et kasutada tika rest teenust.   
 #Võtan kõik allalaetud PDF'id ning muudan need .txt failideks, et mul jääks alles ainult plain text
    for root, dirs, files in os.walk(fold):
        for file in files:
            path_to_pdf = os.path.join(root, file)
            [stem, ext] = os.path.splitext(path_to_pdf)
            if ext == '.pdf':
                print("Processing " + path_to_pdf)
                pdf_contents = parser.from_file(path_to_pdf)

                path_to_txt = stem + '.txt'
                with open(path_to_txt, 'w',encoding="utf-8") as txt_file:
                    print("Writing contents to " + path_to_txt)
                    txt_file.write(pdf_contents['content'])
fetch_pdfs("EE3100073644","2021-03-05")

In [ ]:
quarters = []
basedir = r"C:\Users\marek.keskull\Documents\webscraping"
path = os.path.join(basedir, folder_location)
files = os.listdir(path)
for file in files:
    path_to_pdf = os.path.join(path, file)
    [stem, ext] = os.path.splitext(path_to_pdf)
    if ext == '.txt':
        quarters.append(file)
    

In [ ]:
data = []
for c in quarters:
    with open(path+ "\\" + c, "rb") as file:
        text = file.read()
        asd = text.strip()
        decoded=str(asd,'utf-8')
        data.append(decoded)
data

In [ ]:
#teen listidest dictionary
ddata = dict(zip(quarters, data))
ddata

In [ ]:
def combine_text(list_of_text):
    combined_text = ''.join(list_of_text)
    return combined_text

In [ ]:
data_combined = {key: [combine_text(value)] for (key, value) in ddata.items()}

In [ ]:
#pandas dataframe data
import pandas as pd
pd.set_option('max_colwidth',150)

data_df = pd.DataFrame.from_dict(data_combined).transpose()
data_df.columns = ['text']
data_df = data_df.sort_index()
data_df

# Andmete puhastamine


- Tekst kõik väikeste tähtedega
- Eemdaldame ebavajaliku teksti, mis on kandiliste sulgude vahel
- Eemaldame punktid
- Eemaldame numbrid


In [ ]:
import re
import string

def clean_text_round1(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = re.sub('\w*\d\w*', '', text)
    
    return text

round1 = lambda x: clean_text_round1(x)

In [ ]:
#vaatame puhastatud teksti
data_clean2 = pd.DataFrame(data_df.text.apply(round1))
data_clean2


In [ ]:
# 2. round puhastamisele
def clean_text_round2(text):
    text = re.sub('[‘’“”…]', '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\r', ' ', text)
    text = re.sub(r'\W+', ' ', text)
    return text

round2 = lambda x: clean_text_round2(x)

In [ ]:
#vaatame puhastatud teksti
data_clean2 = pd.DataFrame(data_clean2.text.apply(round2))
pd.options.display.max_colwidth = 500
data_clean2


In [ ]:
import pickle
data_clean2.to_pickle("pickles/lhv2016t2020.pkl")

In [ ]:
#moodustame document term matrix

from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(stop_words='english')
data_cv = cv.fit_transform(data_clean.text)
data_dtm = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names())
data_dtm.index = data_clean.index
data_dtm

In [ ]:
data = data_dtm.transpose()
data.head()

In [ ]:
# top 30 sõna sõnastikus
top_dict = {}
for c in data.columns:
    top = data[c].sort_values(ascending=False).head(30)
    top_dict[c]= list(zip(top.index, top.values))

top_dict

In [ ]:
import pickle
data_dtm.to_pickle("pickles/historicallhvdtm.pkl")